<img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">

# LSEG Data Python Library

**Python Wrapper Package &mdash; Data Retrieval**

Dr. Yves J. Hilpisch | The Python Quants GmbH

<a href="http://tpq.io" target="_blank">http://tpq.io</a> | <a href="http://twitter.com/dyjh" target="_blank">@dyjh</a> | <a href="mailto:training@tpq.io">training@tpq.io</a>

<img src="http://hilpisch.com/images/tr_eikon_02.png" width=350px align=left>

## The Agenda

This tutorial shows

* how to retrieve historical data across asset classes via the LSEG Data Python Library,
* how to work with such data using `pandas`, `Plotly` and `Cufflinks` and
* how to derive support for the Random Walk Hypothesis from financial time series data.

## Random Walk Hypothesis

Eugene F. Fama (1965): “Random Walks in Stock Market Prices”:

> “For many years, economists, statisticians, and teachers of finance have been interested in developing and testing models of stock price behavior. One important model that has evolved from this research is the theory of random walks. This theory casts serious doubt on many other methods for describing and predicting stock price behavior—methods that have considerable popularity outside the academic world. For example, we shall see later that, if the random-walk theory is an accurate description of reality, then the various “technical” or “chartist” procedures for predicting stock prices are completely without value.”

Michael Jensen (1978): “Some Anomalous Evidence Regarding Market Efficiency”:

>“A market is efficient with respect to an information set S if it is impossible to make economic profits by trading on the basis of information set S.”

If a stock price follows a (simple) random walk (no drift & normally distributed returns), then it rises and falls with the same probability of 50% (“toss of a coin”).

**In such a case, the best predictor of tomorrow’s stock price — in a least-squares sense — is today’s stock price.**

## Importing Required Packages

In [126]:
import lseg.data as ld  # the LSEG Python wrapper package
import numpy as np  # NumPy
import pandas as pd  # pandas
import cufflinks as cf  # Cufflinks
import configparser as cp

The following **Python and package versions** are used.

In [127]:
import sys
print(sys.version)

3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)]


In [128]:
ld.__version__

'1.5.1.dev97+g1dedf55e'

In [129]:
np.__version__

'1.23.4'

In [130]:
pd.__version__

'1.5.3'

In [131]:
cf.__version__

'0.17.3'

## Connecting to LSEG's services

The LSEG Data Platform hosts many API Families which themselves host individual APIs which allow users to access data (e.g.: historical end-of-business-day (i.e.: close) prices for a company's publicly floated equity (i.e.: stock)) or services (e.g.: calculations). The [LSEG Data Python Library](https://developers.lseg.com/en) allows us to connect to this platform, and we can authenticate ourselves with the following code:

In [132]:
ld.open_session()

<lseg.data.session.Definition object at 0x24f243c9490 {name='workspace'}>

## Retrieving Cross-Asset Data

We first define a **small universe of `RICS`** for which to retrieve data.

In [133]:
rics = [
    'GE',  # General Electric stock
    'AAPL.O',  # Apple stock
    '.SPX',  # S&P 500 stock index
    '.VIX',  # VIX volatility index
    # 'EUR=',  # EUR/USD exchange rate
    # 'XAU=',  # Gold price
    # 'DE10YT=RR',  # 10yr Bund price
]

Second, **end-of-day (EOD) data** is retrieved.

In [134]:
data = ld.get_history(
    rics,  # the RICs
    fields='TR.PriceClose',  # close field
    start='2024-01-01T12:00:00',  # start date
    end='2024-05-20T12:00:00')  # end date

In [135]:
data.head()  # first five rows

Price Close          GE  AAPL.O     .SPX   .VIX
Date                                           
2023-12-29         <NA>    <NA>  4769.83  12.45
2024-01-02   100.449029  185.64  4742.83   13.2
2024-01-03    98.941696  184.25  4704.81  14.04
2024-01-04    99.420214  181.91  4688.68  14.13
2024-01-05    100.72019  181.18  4697.24  13.35

In [136]:
data.tail()  # final five rows

Price Close      GE  AAPL.O     .SPX   .VIX
Date                                       
2024-05-14    160.0  187.43  5246.68  13.42
2024-05-15   163.47  189.72  5308.15  12.45
2024-05-16   161.12  189.84   5297.1  12.42
2024-05-17   159.89  189.87  5303.27  11.99
2024-05-20   159.28  191.04  5308.13  12.15

Only complete data rows are selected.

In [137]:
data.dropna(inplace=True)  # deletes tows with NaN values

In [138]:
data.info()  # DataFrame meta information

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 97 entries, 2024-01-02 to 2024-05-20
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   GE      97 non-null     Float64
 1   AAPL.O  97 non-null     Float64
 2   .SPX    97 non-null     Float64
 3   .VIX    97 non-null     Float64
dtypes: Float64(4)
memory usage: 4.2 KB


## Calculating the Log Returns

We next calculate the **log returns** in vectorized fashion.

In [139]:
rets = np.log(data / data.shift(1))  # log returns in vectorized fashion

In [140]:
rets.head()

Price Close        GE    AAPL.O      .SPX      .VIX
Date                                               
2024-01-02       <NA>      <NA>      <NA>      <NA>
2024-01-03   -0.01512 -0.007516 -0.008049  0.061694
2024-01-04   0.004825 -0.012781 -0.003434   0.00639
2024-01-05   0.012991 -0.004021  0.001824 -0.056784
2024-01-08   0.015635  0.023887  0.014016 -0.020432

`pandas` allows to derive the **correlation matrix** with a single method call.

In [141]:
data.corr()  # correlation matrix by column

Price Close        GE    AAPL.O      .SPX      .VIX
Price Close                                        
GE           1.000000 -0.554317  0.781073  0.321908
AAPL.O      -0.554317  1.000000 -0.419450 -0.619169
.SPX         0.781073 -0.419450  1.000000 -0.007382
.VIX         0.321908 -0.619169 -0.007382  1.000000

## Plotting the Data

Using `Cufflinks`, we can plot the normalized financial time series as **line plots** for comparison.

In [142]:
cf.set_config_file(offline=True)  # set the plotting mode to offline

In [143]:
data.normalize().iplot(kind='lines')

The frequeny distributions, i.e. the **histograms**, of the log returns per `RIC`.

In [144]:
rets.iplot(kind='histogram', subplots=True)

The **heatmap** below visualizes the correlations between the financial time series.

In [145]:
data.corr().iplot(kind='heatmap', colorscale='blues')

## Preparing Lagged Data

To gain insights into whether the random walk hypothesis holds true, we work with **five lags**. The code that follows derives the **lagged data** for every single `RIC`. First, a function that adds columns with lagged data to a `DataFrame` object.

In [146]:
def add_lags(data, ric, lags):
    cols = []
    df = pd.DataFrame(data[ric])
    for lag in range(1, lags + 1):
        col = 'lag_{}'.format(lag)  # defines the column name
        df[col] = df[ric].shift(lag)  # creates the lagged data column
        cols.append(col)  # stores the column name
    df.dropna(inplace=True)  # gets rid of incomplete data rows
    return df, cols

Second, the iterations over all `RICs`, using the `add_lags` function and storing the resulting `DataFrame` objects in a dictonary.

In [147]:
lags = 5  # five historical lags

In [148]:
dfs = {}
for ric in rics:
    df, cols = add_lags(data, ric, lags)
    dfs[ric] = df

In [149]:
cols  # the column names for the lags

['lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5']

In [150]:
dfs.keys()  # the keys of the dictonary

dict_keys(['GE', 'AAPL.O', '.SPX', '.VIX'])

In [151]:
dfs['AAPL.O'].head(7)

AAPL.O   lag_1   lag_2   lag_3   lag_4   lag_5
Date                                                      
2024-01-09  185.14  185.56  181.18  181.91  184.25  185.64
2024-01-10  186.19  185.14  185.56  181.18  181.91  184.25
2024-01-11  185.59  186.19  185.14  185.56  181.18  181.91
2024-01-12  185.92  185.59  186.19  185.14  185.56  181.18
2024-01-16  183.63  185.92  185.59  186.19  185.14  185.56
2024-01-17  182.68  183.63  185.92  185.59  186.19  185.14
2024-01-18  188.63  182.68  183.63  185.92  185.59  186.19

## Implementing OLS Regression

The matrix consisting of the lagged data columns is used to "predict" the next day's value of the `RIC` via **linear OLS regression**.

In [216]:
regs = {}
for ric in rics:
    df = dfs[ric]  # getting data for the RIC
    
    # tidying the data to fit the type nessesary for our OLS Python function `np.linalg.lstsq`
    x = np.vstack([float(i) for i in df[ric].values])
    A = np.vstack([
        np.array([float(j) for j in i]) for i in [
            row[1].values for row in df[cols].iterrows()]])
    
    reg = np.linalg.lstsq(A, x, rcond=None)[0]  # the OLS regression
    regs[ric] = np.array([i[0] for i in reg])  # storing the results

In [218]:
for ric in rics:
    print('{:10} | {}'.format(ric, regs[ric]))

GE         | [ 0.96099202 -0.04616513 -0.11861937  0.31735471 -0.10811975]
AAPL.O     | [ 1.08669392 -0.10726631 -0.01355608  0.05522921 -0.02087889]
.SPX       | [ 1.00869959 -0.24003522  0.23023117  0.1030741  -0.10064285]
.VIX       | [ 1.04309786 -0.21866963  0.1325518   0.07127188 -0.0302332 ]


## Taking a Closer Look

Let's pick one `RIC` and compare the original time series with the OLS predicted one.

In [219]:
ric = 'AAPL.O'

In [220]:
res = pd.DataFrame(dfs[ric][ric])  # picks the original time series

In [221]:
res['PRED'] = np.dot(dfs[ric][cols], regs[ric])  # creates the "prediction" values

The **predicted prices** are almost exactly the prices from the day before.

In [222]:
res.iloc[-50:].iplot()

In [223]:
res.head()

AAPL.O        PRED
Date                          
2024-01-09  185.14  186.046453
2024-01-10  186.19  185.029897
2024-01-11  185.59  186.165141
2024-01-12  185.92  185.663334
2024-01-16  183.63  185.957423

## Analyzing the Results

Now analyzing the **regression results** a bit more formally.

In [224]:
rega = np.stack(regs.values())  # combines the regression results
rega

C:\Users\U6082174\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3488: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



array([[ 0.96099202, -0.04616513, -0.11861937,  0.31735471, -0.10811975],
       [ 1.08669392, -0.10726631, -0.01355608,  0.05522921, -0.02087889],
       [ 1.00869959, -0.24003522,  0.23023117,  0.1030741 , -0.10064285],
       [ 1.04309786, -0.21866963,  0.1325518 ,  0.07127188, -0.0302332 ]])

Almost all the weight lies on the most recent price (`lag_1`).

In [225]:
rega.mean(axis=0)  # mean values by column

array([ 1.02487085, -0.15303407,  0.05765188,  0.13673247, -0.06496867])

In [226]:
regd = pd.DataFrame(rega, columns=cols, index=rics)  # converting the results to DataFrame

In [227]:
regd

lag_1     lag_2     lag_3     lag_4     lag_5
GE      0.960992 -0.046165 -0.118619  0.317355 -0.108120
AAPL.O  1.086694 -0.107266 -0.013556  0.055229 -0.020879
.SPX    1.008700 -0.240035  0.230231  0.103074 -0.100643
.VIX    1.043098 -0.218670  0.132552  0.071272 -0.030233

In [228]:
regd.describe()  # summary statistics

lag_1     lag_2     lag_3     lag_4     lag_5
count  4.000000  4.000000  4.000000  4.000000  4.000000
mean   1.024871 -0.153034  0.057652  0.136732 -0.064969
std    0.053218  0.092002  0.154419  0.122045  0.045772
min    0.960992 -0.240035 -0.118619  0.055229 -0.108120
25%    0.996773 -0.224011 -0.039822  0.067261 -0.102512
50%    1.025899 -0.162968  0.059498  0.087173 -0.065438
75%    1.053997 -0.091991  0.156972  0.156644 -0.027895
max    1.086694 -0.046165  0.230231  0.317355 -0.020879

## Visualizing the Results

The following bar chart illustrates that the results a qualitatively similar for all `RICS` analyzed &mdash; "_today's price is the best predictor, in a least-squares sense, for tomorrow's price_".

In [229]:
regd.iplot(kind='bar')

The **mean values** for the single optimal regression parameters.

In [230]:
regd.mean().iplot(kind='bar')

## Analyzing Intraday Data

Let us quickly check, whether the results are similar on an **intraday basis**.

In [242]:
data = ld.get_history(
    rics[0:2],  # RICs
    fields='BID',  # fields to be retrieved
    start='2024-05-20T14:00:00',  # start date
    end='2024-05-20T18:00:00',  # end date
    interval='minute')  # bar length

In [243]:
data

BID                      GE  AAPL.O
Timestamp                          
2024-05-20 14:00:00   160.5  190.93
2024-05-20 14:01:00  160.51  190.83
2024-05-20 14:02:00   160.4  190.92
2024-05-20 14:03:00  160.58  191.08
2024-05-20 14:04:00   160.6  191.28
...                     ...     ...
2024-05-20 17:55:00   159.9  191.45
2024-05-20 17:56:00  159.88  191.43
2024-05-20 17:57:00   159.9  191.45
2024-05-20 17:58:00  159.96  191.47
2024-05-20 17:59:00  160.03   191.5

[240 rows x 2 columns]

In [244]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 240 entries, 2024-05-20 14:00:00 to 2024-05-20 17:59:00
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   GE      240 non-null    Float64
 1   AAPL.O  240 non-null    Float64
dtypes: Float64(2)
memory usage: 6.1 KB


In [245]:
data.tail()

BID                      GE  AAPL.O
Timestamp                          
2024-05-20 17:55:00   159.9  191.45
2024-05-20 17:56:00  159.88  191.43
2024-05-20 17:57:00   159.9  191.45
2024-05-20 17:58:00  159.96  191.47
2024-05-20 17:59:00  160.03   191.5

In [250]:
dfs = {}
for ric in rics[0:2]:
    df, cols = add_lags(data, ric, lags)
    dfs[ric] = df

In [252]:
regs = {}
for ric in rics[0:2]:
    df = dfs[ric]
    
    # tidying the data to fit the type nessesary for our OLS Python function `np.linalg.lstsq`
    x = np.vstack([float(i) for i in df[ric].values])
    A = np.vstack([
        np.array([float(j) for j in i]) for i in [
            row[1].values for row in df[cols].iterrows()]])
    
    reg = np.linalg.lstsq(A, x, rcond=None)[0]  # the OLS regression
    regs[ric] = np.array([i[0] for i in reg])  # storing the results

In [253]:
# regs = {}
# for ric in rics[0:2]:
#     df = dfs[ric]
#     reg = np.linalg.lstsq(df[cols], df[ric], rcond=-1)[0]
#     regs[ric] = reg

In [254]:
rega = np.stack(regs.values())

C:\Users\U6082174\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3488: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



In [255]:
regd = pd.DataFrame(rega, columns=cols, index=rics[0:2])

**Intraday** the optimal regression parameters show more variation.

In [256]:
regd.iplot(kind='bar')

In [257]:
regd.mean().iplot(kind='bar')

## Conclusions

Based on this tutorial, we can conclude that

* it is easy to retrieve **historical end-of-day and intraday data across asset classes** via the LSEG Data Python Library,
* `Plotly` and `Cufflinks` make **financial data visualization** convenient and
* there is **support for the Random Walk Hypothesis** based on the OLS regression analysis (both daily and a little bit less so intraday).

## LSEG Data Developer Resources

* [API Pages](https://developers.lseg.com/en/api-catalog) 
* [Q&A Forums](https://community.developers.refinitiv.com/index.html) 

Data Item Browser Application: Type [`DIB`](https://developers.lseg.com/en/video-catalog/data-item-browser) into Workspace Search Bar.

<img src="https://developers.lseg.com/content/dam/devportal/icons/logo/lseg-logo.svg" width="28%" align="left" style="vertical-align: top; padding-top: 23px;">
<img src="http://hilpisch.com/tpq_logo_long.png" width="36%" align="right" style="vertical-align: top;">